<a href="https://colab.research.google.com/github/audalsgh/20250730/blob/main/0730_Peoplenet%2BTrafficnet%2BYOLOv11_ensemble.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install yt-dlp
!pip install onnxruntime-gpu  # GPU 지원
!pip install opencv-python-headless numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.4/175.4 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 107.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.2/283.2 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 9.5 MB/s eta 0:00:00


In [2]:
# 직접 업로드
from google.colab import files
uploaded = files.upload()  # 여기서 peoplenet.onnx 선택

Saving resnet34_peoplenet_int8.onnx to resnet34_peoplenet_int8.onnx


In [3]:
import onnxruntime as ort
import cv2
import numpy as np
import glob

# 모델 경로 지정
model_path = '/content/resnet34_peoplenet_int8.onnx'  # (A)의 경우 업로드한 파일명
# model_path = '/content/drive/MyDrive/models/peoplenet.onnx'  # (B) Drive 경로

# ONNX InferenceSession 생성 (CUDA, fallback to CPU)
providers = ['CUDAExecutionProvider', 'CPUExecutionProvider']
session = ort.InferenceSession(model_path, providers=providers)

# 입력 이름 확인
input_name = session.get_inputs()[0].name
print(f"✅ ONNX 모델 로드 완료. Input Name: {input_name}")

✅ ONNX 모델 로드 완료. Input Name: input_1:0


In [4]:
# 1) yt-dlp 설치 (이미 설치했다면 스킵)
!pip install -q yt-dlp

# 2) 유튜브 영상 다운로드
import yt_dlp

YOUTUBE_URL = 'https://www.youtube.com/watch?v=J71x_FW1Mdg'
OUTPUT_FILENAME = 'video.mp4'

ydl_opts = {
    'format': 'mp4',
    'outtmpl': OUTPUT_FILENAME,
    # 필요시 쿠키나 헤더 옵션 추가 가능
}

with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    print(f"▶️ 다운로드 시작: {YOUTUBE_URL}")
    ydl.download([YOUTUBE_URL])
    print(f"✅ 다운로드 완료: {OUTPUT_FILENAME}")

# 3) video_path 변수에 할당
video_path = OUTPUT_FILENAME
print(f"🎬 사용할 비디오 경로: {video_path}")

▶️ 다운로드 시작: https://www.youtube.com/watch?v=J71x_FW1Mdg
[youtube] Extracting URL: https://www.youtube.com/watch?v=J71x_FW1Mdg
[youtube] J71x_FW1Mdg: Downloading webpage
[youtube] J71x_FW1Mdg: Downloading tv client config
[youtube] J71x_FW1Mdg: Downloading player 0b00c3eb-main
[youtube] J71x_FW1Mdg: Downloading tv player API JSON
[youtube] J71x_FW1Mdg: Downloading ios player API JSON
[youtube] J71x_FW1Mdg: Downloading m3u8 information
[info] J71x_FW1Mdg: Downloading 1 format(s): 18
[download] Destination: video.mp4
[download] 100% of    5.49MiB in 00:00:02 at 2.38MiB/s   
✅ 다운로드 완료: video.mp4
🎬 사용할 비디오 경로: video.mp4


In [5]:
# 4) ONNX 입력 메타데이터로 기대 입력 크기 확인
input_meta = session.get_inputs()[0]
_, C, H, W = input_meta.shape
print(f"Model expects input shape (N, C, H, W): (1, {C}, {H}, {W})")

# 5) OpenCV로 비디오 열기 및 VideoWriter 초기화
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    raise RuntimeError(f"동영상을 열 수 없습니다: {video_path}")

fps    = cap.get(cv2.CAP_PROP_FPS)
width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out    = cv2.VideoWriter('output.mp4', fourcc, fps, (width, height))

print(f"▶️ 추론 시작: fps={fps}, size=({width},{height})")

# 6) 프레임별 추론 및 비디오 기록 (전체 채널 합산 + closing)
import cv2
import numpy as np

frame_idx = 0
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # 전처리
    img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (W, H))
    img = img.astype(np.float32) / 255.0
    img = np.transpose(img, (2,0,1))[None, ...]

    # ONNX 추론
    outputs = session.run(None, {input_meta.name: img})
    heatmaps = outputs[0][0]           # (3, H, W)

    # ❗️ 모든 채널 합산 (head+torso+legs)
    heatmap = np.sum(heatmaps, axis=0)  # (H, W)

    # 원본 프레임 크기에 맞춰 리사이즈 & 정규화(0~255) & uint8
    mask = cv2.resize(heatmap, (width, height))
    mask_uint8 = (mask * 255).clip(0,255).astype(np.uint8)

    # Morphological closing으로 작은 구멍 메우기
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7,7))
    mask_closed = cv2.morphologyEx(mask_uint8, cv2.MORPH_CLOSE, kernel)

    # 이진 임계처리 (값 분포에 따라 100~150 사이로 조절)
    _, bin_mask = cv2.threshold(mask_closed, 120, 255, cv2.THRESH_BINARY)

    # 컨투어 추출 및 박스 그리기
    contours, _ = cv2.findContours(bin_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for cnt in contours:
        x,y,w,h = cv2.boundingRect(cnt)
        if w*h < 500:  # 노이즈 필터
            continue
        cv2.rectangle(frame, (x,y), (x+w,y+h), (0,255,0), 2)

    # 프레임 번호 표시
    cv2.putText(frame,
                f"Frame {frame_idx}",
                (10,30),
                cv2.FONT_HERSHEY_SIMPLEX,
                1, (0,255,0), 2)

    out.write(frame)
    frame_idx += 1

cap.release()
out.release()
print(f"✅ 처리 완료: output.mp4 (총 {frame_idx} 프레임)")

# 7) 피플넷 결과 비디오 다운로드
from google.colab import files
files.download('output.mp4')

Model expects input shape (N, C, H, W): (1, 3, 544, 960)
▶️ 추론 시작: fps=29.97002997002997, size=(640,360)
✅ 처리 완료: output.mp4 (총 2786 프레임)


In [19]:
# ──────────────────────────────────────────────────
# 📦 TrafficNet 역할 대체: YOLO로 교통 객체만 검출
# ──────────────────────────────────────────────────

# 1) 라이브러리 설치 (한 번만)
!pip install -q ultralytics opencv-python-headless yt-dlp

from ultralytics import YOLO
import cv2

# 2) 모델 로드 (yolov11n 자동 다운로드)
model = YOLO("yolo11n")

# 3) 교통 객체 클래스 인덱스 (COCO 기준)
TRAFFIC_CLASSES = [2, 3, 5, 7, 9, 11]  # car, motorcycle, bus, truck, traffic light, stop sign

# 4) 영상 열기 (이미 다운로드된 video.mp4 사용)
cap = cv2.VideoCapture("video.mp4")
fps      = cap.get(cv2.CAP_PROP_FPS) or 1
interval = max(1, int(fps))
w  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h  = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out_traffic = cv2.VideoWriter("trafficnet.mp4", fourcc, fps, (w, h))

idx = 0
while True:
    ret, frame = cap.read()
    if not ret:
        break

    if idx % interval == 0:
        # classes 필터링 옵션으로 교통 객체만 검출
        res = model.predict(frame, classes=TRAFFIC_CLASSES, conf=0.25, verbose=False)[0]
        for box, conf, cls in zip(res.boxes.xyxy, res.boxes.conf, res.boxes.cls):
            x1, y1, x2, y2 = map(int, box)
            label = f"{model.names[int(cls)]} {conf:.2f}"
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)
            cv2.putText(frame, label, (x1, y1 - 5),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 1)

    out_traffic.write(frame)
    idx += 1

cap.release()
out_traffic.release()
print("✅ TrafficNet 역할 영상 저장: trafficnet.mp4")

100%|██████████| 5.35M/5.35M [00:00<00:00, 330MB/s]


✅ TrafficNet 역할 영상 저장: trafficnet.mp4


In [24]:
# ──────────────────────────────────────────────────
# 📦 YOLOv11n (ultralytics) 로드 & 추론
# ──────────────────────────────────────────────────

# 1) 라이브러리 설치 (한 번만 실행)
!pip install -q ultralytics opencv-python-headless yt-dlp

from ultralytics import YOLO
import cv2

# 2) 모델 로드 (yolov11n 자동 다운로드)
model = YOLO("yolo11n")
print("✅ YOLOv11n 모델 로드됨")

# 3) 비디오 열기 / 쓰기 준비
VIDEO_PATH = "video.mp4"  # 앞에서 다운받은 파일
cap    = cv2.VideoCapture(VIDEO_PATH)
fps    = cap.get(cv2.CAP_PROP_FPS) or 1
interval = max(1, int(fps))  # 1초당 1프레임
w  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h  = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out    = cv2.VideoWriter("yolov11n_detect.mp4", fourcc, fps, (w, h))

# 4) 추론 루프
idx = 0
while True:
    ret, frame = cap.read()
    if not ret:
        break

    if idx % interval == 0:
        # ultralytics YOLO 추론 (confidence 0.25)
        res = model.predict(frame, conf=0.25, verbose=False)[0]
        for box, conf, cls in zip(res.boxes.xyxy, res.boxes.conf, res.boxes.cls):
            x1, y1, x2, y2 = map(int, box)
            label = f"{model.names[int(cls)]} {conf:.2f}"
            cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
            cv2.putText(frame, label, (x1, y1 - 5),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,0,0), 1)

    out.write(frame)
    idx += 1

cap.release()
out.release()
print(f"✅ YOLOv11n 결과 저장: yolov11n_detect.mp4 (총 {idx} 프레임 처리)")

✅ YOLOv11n 모델 로드됨
✅ YOLOv11n 결과 저장: yolov11n_detect.mp4 (총 2786 프레임 처리)


In [28]:
# ──────────────────────────────────────────────────
# 📦 PeopleNetWrapper 정의 및 인스턴스화
# ──────────────────────────────────────────────────

import onnxruntime as ort
import cv2
import numpy as np

# 1) 이미 ONNX Runtime 세션을 생성했다면 이 줄을 주석 해제하세요:
# providers = ['CUDAExecutionProvider','CPUExecutionProvider']
# session   = ort.InferenceSession(model_path, providers=providers)

# 2) PeopleNetWrapper 클래스 정의
class PeopleNetWrapper:
    def __init__(self, session, conf_th=0.5, area_th=500):
        self.session    = session
        meta            = session.get_inputs()[0]
        self.input_name = meta.name
        _, C, self.H, self.W = meta.shape
        self.conf_th    = conf_th
        self.area_th    = area_th

    def preprocess(self, frame):
        img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (self.W, self.H)).astype(np.float32) / 255.0
        return np.transpose(img, (2,0,1))[None, ...]

    def postprocess(self, outs, orig_shape):
        heatmap = np.sum(outs[0][0][..., 4], axis=0)
        mask    = cv2.resize(heatmap, (orig_shape[1], orig_shape[0]))
        _, bm   = cv2.threshold((mask*255).astype(np.uint8),
                                int(self.conf_th*255), 255, cv2.THRESH_BINARY)
        dets = []
        cnts,_ = cv2.findContours(bm, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        for c in cnts:
            x,y,w,h = cv2.boundingRect(c)
            if w*h < self.area_th: continue
            dets.append({
                'bbox':       [x, y, x+w, y+h],
                'confidence': float(bm[y:y+h, x:x+w].mean()/255.0),
                'class':      'person'
            })
        return dets

    def detect_people(self, frame):
        inp  = self.preprocess(frame)
        outs = self.session.run(None, {self.input_name: inp})
        return self.postprocess(outs, frame.shape[:2])

# 3) real_people_net 인스턴스 생성
real_people_net = PeopleNetWrapper(session)
print("✅ real_people_net 인스턴스가 생성되었습니다.")

✅ real_people_net 인스턴스가 생성되었습니다.


In [35]:
# ──────────────────────────────────────────────────
# 🎲 앙상블: WBF으로 PeopleNet + Traffic(=YOLO filter) + YOLOv11n 결합
# ──────────────────────────────────────────────────

# 1) 설치 (한번만 실행)
!pip install -q ensemble-boxes ultralytics opencv-python-headless

from ensemble_boxes import weighted_boxes_fusion
from ultralytics import YOLO
import cv2
import numpy as np

# 2) PeopleNet 래퍼 인스턴스 (이미 정의/생성됨)
pn = real_people_net

# 3) YOLOv11n 모델 로드 (두 용도로 각각)
traffic_model = YOLO("yolo11n")   # 교통 객체 필터링
yolo_model    = YOLO("yolo11n")   # 일반 객체

# 4) 클래스→ID 맵핑 준비
class_to_id = {'person': 0, 'traffic': 1}

# COCO 80개 클래스 모두 등록 (2~81)
for i, name in enumerate(yolo_model.names):
    class_to_id[name] = i + 2

# 5) 동영상 열기/쓰기 세팅
VIDEO_PATH = "video.mp4"
cap    = cv2.VideoCapture(VIDEO_PATH)
fps    = cap.get(cv2.CAP_PROP_FPS) or 1
interval = max(1, int(fps))
w  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h  = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
out  = cv2.VideoWriter("ensemble.mp4", cv2.VideoWriter_fourcc(*"mp4v"), fps, (w,h))

# 6) 앙상블 루프
idx = 0
while True:
    ret, frame = cap.read()
    if not ret: break

    if idx % interval == 0:
        boxes_list, scores_list, labels_list = [], [], []

        # 6.1 PeopleNet
        pn_dets = pn.detect_people(frame)
        b1, s1, l1 = [], [], []
        for d in pn_dets:
            x1,y1,x2,y2 = d['bbox']
            b1.append([x1/w, y1/h, x2/w, y2/h])
            s1.append(d['confidence'])
            l1.append(class_to_id['person'])
        boxes_list.append(b1); scores_list.append(s1); labels_list.append(l1)

        # 6.2 TrafficNet 역할 (교통 클래스만)
        TRAFFIC_CLASSES = [2,3,5,7,9,11]
        res_t = traffic_model.predict(frame, classes=TRAFFIC_CLASSES, conf=0.25, verbose=False)[0]
        b2, s2, l2 = [], [], []
        for box, conf, cls in zip(res_t.boxes.xyxy, res_t.boxes.conf, res_t.boxes.cls):
            x1,y1,x2,y2 = map(int, box)
            b2.append([x1/w, y1/h, x2/w, y2/h])
            s2.append(float(conf))
            l2.append(class_to_id['traffic'])
        boxes_list.append(b2); scores_list.append(s2); labels_list.append(l2)

        # 6.3 YOLOv11n 전체 객체
        res_y = yolo_model.predict(frame, conf=0.25, verbose=False)[0]
        b3, s3, l3 = [], [], []
        for box, conf, cls in zip(res_y.boxes.xyxy, res_y.boxes.conf, res_y.boxes.cls):
            name = yolo_model.names[int(cls)]
            # 동적 매핑: 혹시 누락된 클래스가 있다면 추가
            if name not in class_to_id:
                class_to_id[name] = len(class_to_id)
            x1,y1,x2,y2 = map(int, box)
            b3.append([x1/w, y1/h, x2/w, y2/h])
            s3.append(float(conf))
            l3.append(class_to_id[name])
        boxes_list.append(b3); scores_list.append(s3); labels_list.append(l3)

        # 6.4 WBF 적용
        if any(boxes_list):
            fb, fs, fl = weighted_boxes_fusion(
                boxes_list, scores_list, labels_list,
                iou_thr=0.5, skip_box_thr=0.3
            )
            for box, score, label in zip(fb, fs, fl):
                x1 = int(box[0]*w); y1 = int(box[1]*h)
                x2 = int(box[2]*w); y2 = int(box[3]*h)
                # 색상: person=녹, traffic=빨, 기타=파랑
                color = (0,255,0) if label==class_to_id['person'] else \
                        (0,0,255) if label==class_to_id['traffic'] else (255,0,0)
                # label→name 역매핑
                name = [k for k,v in class_to_id.items() if v==label][0]
                cv2.rectangle(frame, (x1,y1), (x2,y2), color, 2)
                cv2.putText(frame, f"{name} {score:.2f}", (x1, y1-5),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)

    out.write(frame)
    idx += 1

cap.release()
out.release()
print(f"✅ 앙상블 완료: ensemble.mp4 (총 {idx} 프레임 처리)")

✅ 앙상블 완료: ensemble.mp4 (총 2786 프레임 처리)
